# Creación de Consultas para los Endpoints de la API:

#### 1. Importar librerías y archivos necesarios:

In [1]:
import pandas as pd

# Steam Games:
SGames = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')

# Users Items:
UItems = pd.read_csv('Clean_Data\\UItems_CD.csv', quotechar='"')

# Users Reviews:
UReviews = pd.read_csv('Clean_Data\\UReviews_CD.csv', quotechar='"')

# Users Reviews:
Genres_Items = pd.read_csv('Clean_Data\Genres_Items.csv', quotechar='"')

### 2. Crear la formula registros por desarrollador:

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

| Año  | Cantidad de artículos | Contenido Gratis |
|------|------------------------|-------------------|
| 2023 | 50                     | 27%               |
| 2022 | 45                     | 25%               |
| xxxx | xx                     | x%                |


2.1. Crear datos de entrada:

In [2]:
# Seleccionar las columnas deseadas y eliminar filas con datos nulos
End1_Developer = SGames[['id', 'developer', 'release_year', 'price']].dropna()

# Mostrar el DataFrame resultante
print("\nDataFrame final:")
End1_Developer



DataFrame final:


,id,developer,release_year,price
0,761140.0,Kotoshiro,2018,4.99
1,643980.0,Secret Level SRL,2018,0.00
2,670290.0,Poolians.com,2017,0.00
3,767400.0,彼岸领域,2017,0.99
5,772540.0,Trickjump Games Ltd,2018,3.99
...,...,...,...,...
32127,745400.0,Bidoniera Games,2018,1.99
32128,773640.0,"Nikita ""Ghost_RUS""",2018,1.99
32129,733530.0,Sacada,2018,4.99
32130,610660.0,Laush Dmitriy Sergeevich,2018,1.99


2.2. Obtener top-five de desarrolladores con más items para probar la fórmula:

In [3]:
# Calcular el top 5 de desarrolladores que más aparecen
top_developers = End1_Developer['developer'].value_counts().head(-5)

# Mostrar el top 5
print("Top 5 de desarrolladores que más aparecen:")
print(top_developers)

Top 5 de desarrolladores que más aparecen:
developer
Ubisoft - San Francisco       1258
SmiteWorks USA, LLC            808
Dovetail Games                 252
KOEI TECMO GAMES CO., LTD.     214
Paradox Development Studio     150
                              ... 
O'Stinky lab.                    1
Attila D.                        1
Hont                             1
Xform                            1
Gustav                           1
Name: count, Length: 10599, dtype: int64


2.3. Crear fórmula "get_developer_stats":

In [4]:
def get_developer_stats(developer):
    
    # Filtrar el DataFrame por desarrollador
    developer_df = End1_Developer[End1_Developer['developer'] == developer]

    # Calcular la cantidad de items y el porcentaje de contenido gratuito por año
    stats_by_year = developer_df.groupby('release_year').agg({'id': 'count', 'price': lambda x: (x == 0).sum() / len(x) * 100}).reset_index()
   
    stats_by_year.columns = ['release_year', 'item_count', 'free_content_percentage']

    # Convertir los resultados a una lista de diccionarios
    result_list = stats_by_year.to_dict(orient='records')

    return result_list



2.4. Probar la fórmula:

In [5]:
# Ejemplo de uso de la función
print(get_developer_stats('Secret Level SRL'))

print(get_developer_stats('KOEI TECMO GAMES CO., LTD.'))

[{'release_year': 2018, 'item_count': 1, 'free_content_percentage': 100.0}]
[{'release_year': 0, 'item_count': 1, 'free_content_percentage': 0.0}, {'release_year': 2014, 'item_count': 28, 'free_content_percentage': 0.0}, {'release_year': 2015, 'item_count': 53, 'free_content_percentage': 0.0}, {'release_year': 2016, 'item_count': 5, 'free_content_percentage': 0.0}, {'release_year': 2017, 'item_count': 127, 'free_content_percentage': 7.086614173228346}]


2.5. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [6]:
# Exportar el DataFrame a CSV
End1_Developer.to_csv('Data_Queries/End1_Developer.csv', index=False)

print(f"DataFrame exportado a {'Data_Queries/End1_Developer.csv'}")

DataFrame exportado a Data_Queries/End1_Developer.csv


### 3. Crear función registros de usuario:

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

3.1. Crear datos de entrada:

In [58]:

# Crear el DataFrame End2_User
UItems_F= UItems[['user_id', 'item_id']]
End1_Developer_F=End1_Developer[['id','price']]
End2_User = pd.merge(UItems_F, End1_Developer, left_on='item_id', right_on='id')
End2_User = End2_User[['id', 'user_id', 'price']]
UReviews_F = UReviews[['item_id', 'user_id','recommend']]


# Mostrar el DataFrame resultante
print("\nDataFrame End2_User:")
print(End2_User.head(3))

print("\nDataFrame UReviews_F:")
print(UReviews_F.head(3))


DataFrame End2_User:
     id            user_id  price
0  10.0  76561197970982479   9.99
1  10.0            js41637   9.99
2  10.0         Riot-Punch   9.99

DataFrame UReviews_F:
   item_id            user_id  recommend
0     1250  76561197970982479       True
1    22200  76561197970982479       True
2    43110  76561197970982479       True


3.2. Crear fórmula "userdata":

In [65]:
def userdata(User_id):
    # Filtrar el DataFrame End2_User por user_id
    user_df = End2_User[End2_User['user_id'] == User_id]

    # Calcular la cantidad de dinero gastado por el usuario
    total_money_spent = user_df['price'].sum()

    # Calcular la cantidad de items
    total_items = len(user_df)

    # Filtrar el DataFrame UReviews_F por item_ids relacionados con el usuario
    user_reviews_df = UReviews_F[UReviews_F['user_id'] == User_id]

    # Calcular el porcentaje de recomendación
    total_recommendations = user_reviews_df['recommend'].sum() # No es necesario especificar "recommend = True"
    recommendation_percentage = (total_recommendations / total_items) * 100 if total_items > 0 else 0

    # Crear el diccionario de resultados
    result_dict = {
        "Usuario": User_id,
        "Dinero gastado": f"{total_money_spent} USD",
        "% de recomendación": f"{recommendation_percentage:.2f}%",
        "Cantidad de items": total_items
    }

    return result_dict

3.3. Probar fórmula:

In [63]:
# Ejemplo de uso de la función
user_stats_result = userdata('76561197970982479')
print(user_stats_result)

{'Usuario': '76561197970982479', 'Dinero gastado': '3404.33 USD', '% de recomendación': '1.33%', 'Cantidad de items': 226}


3.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [66]:
# Exportar el DataFrame a CSV:

End2_User.to_csv('Data_Queries/End2_User.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/End2_User.csv'}")

UReviews_F.to_csv('Data_Queries/UReviews_F.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/UReviews_F.csv'}")

DataFrame exportado a Data_Queries/End2_User.csv
DataFrame exportado a Data_Queries/UReviews_F.csv


### 4. Función Usuario por Género de video Juego:

def UserForGenre( genero: str ) : Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas : 23}]}

4.1. Crear datos de Entrada:

In [95]:
# crear subset de SGames con las columnas necesarias:
subset_SGames = SGames[['id', 'release_year']]

# Unir subset_SGames y Genres_Items por la columna 'id':
merged_data = pd.merge(subset_SGames, Genres_Items, on='id')
merged_data

,id,release_year,genres
0,761140.0,2018,Action
1,761140.0,2018,Casual
2,761140.0,2018,Indie
3,761140.0,2018,Simulation
4,761140.0,2018,Strategy
...,...,...,...
80404,681550.0,0,Adventure
80405,681550.0,0,Indie
80406,681550.0,0,Action
80407,681550.0,0,Simulation


In [96]:
# Seleccionar las columnas 'user_id', 'item_id' y 'playtime_forever' de UItems_CD:
UItems_subset = UItems[['user_id', 'item_id', 'playtime_forever']]

# Unir UItems_CD_subset con merged_data por las columnas 'item_id' e 'id':
End3_Genre_User = pd.merge(UItems_subset, merged_data, left_on='item_id', right_on='id')

# Eliminar la columna duplicada 'id':
End3_Genre_User = End3_Genre_User.drop(columns=['id'])

# Eliminar filas con Año==0 en UserForGenre:
End3_Genre_User = End3_Genre_User[End3_Genre_User['release_year'] != 0]

# Mostrar el DataFrame resultante:
End3_Genre_User.head()


,user_id,item_id,playtime_forever,release_year,genres
0,76561197970982479,10,6,2000,Action
1,js41637,10,0,2000,Action
2,Riot-Punch,10,0,2000,Action
3,doctr,10,93,2000,Action
4,corrupted_soul,10,108,2000,Action


4.2. Crear función UserForGenre:

In [98]:
def UserForGenre(genero: str):
    # Filtrar UserForGenre por el género dado
    genre_data = End3_Genre_User[End3_Genre_User['genres'] == genero]

    # Encontrar al usuario con más horas jugadas para el género dado
    max_hours_user = genre_data.loc[genre_data['playtime_forever'].idxmax()]['user_id']

    # Crear una lista de la acumulación de horas jugadas por año de lanzamiento
    hours_by_year = genre_data.groupby('release_year')['playtime_forever'].sum().reset_index()
    hours_by_year_list = [{"Año": int(year), "Horas": int(hours)} for year, hours in hours_by_year.values]

    # Crear el diccionario de resultados
    result_dict = {
        f"Usuario con más horas jugadas para el género {genero}": max_hours_user,
        "Horas jugadas": hours_by_year_list
    }

    return result_dict


4.3. Probar función:

In [99]:
# Ejemplo de uso
genero_ejemplo = "Action"  # Reemplazar con el género deseado
resultado = UserForGenre(genero_ejemplo)
print(resultado)

{'Usuario con más horas jugadas para el género Action': 'Evilutional', 'Horas jugadas': [{'Año': 1983, 'Horas': 3582}, {'Año': 1984, 'Horas': 384}, {'Año': 1988, 'Horas': 16243}, {'Año': 1989, 'Horas': 607}, {'Año': 1990, 'Horas': 18787}, {'Año': 1991, 'Horas': 2502}, {'Año': 1992, 'Horas': 1925}, {'Año': 1993, 'Horas': 24663}, {'Año': 1994, 'Horas': 121057}, {'Año': 1995, 'Horas': 217813}, {'Año': 1996, 'Horas': 70061}, {'Año': 1997, 'Horas': 363033}, {'Año': 1998, 'Horas': 2935053}, {'Año': 1999, 'Horas': 2735338}, {'Año': 2000, 'Horas': 18661379}, {'Año': 2001, 'Horas': 1337303}, {'Año': 2002, 'Horas': 740382}, {'Año': 2003, 'Horas': 11969297}, {'Año': 2004, 'Horas': 131222294}, {'Año': 2005, 'Horas': 12426135}, {'Año': 2006, 'Horas': 10941224}, {'Año': 2007, 'Horas': 17744979}, {'Año': 2008, 'Horas': 33990368}, {'Año': 2009, 'Horas': 165437678}, {'Año': 2010, 'Horas': 182666321}, {'Año': 2011, 'Horas': 335982600}, {'Año': 2012, 'Horas': 1099231451}, {'Año': 2013, 'Horas': 420745701

4.5. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [100]:
# Exportar el DataFrame a CSV:

End3_Genre_User.to_csv('Data_Queries/End3_Genre_User.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/End3_Genre_User.csv'}")


DataFrame exportado a Data_Queries/End3_Genre_User.csv


### 5. Crear función de mejor desarrollador por año:

def best_developer_year( año: int ) : Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = Verdadero y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]


5.1. Crear los datos de entrada:

In [101]:
# Filtrar por recommend == True y sentiment_analysis == 2
filtered_reviews = UReviews[(UReviews['recommend'] == True) & (UReviews['sentiment_analysis'] == 2)]

# Resumir por el conteo de "item_id" por cada año "posted_year"
UReviews_gb = filtered_reviews.groupby(['posted_year', 'item_id']).size().reset_index(name='count')

# Mostrar el DataFrame resultante
UReviews_gb


,posted_year,item_id,count
0,2010,240,2
1,2010,300,1
2,2010,400,2
3,2010,440,9
4,2010,550,1
...,...,...,...
3842,2015,419070,1
3843,2015,421630,1
3844,2015,423120,1
3845,2015,423880,7


Solo se requiere saber el desarrollador y para ello se debe filtrar el dataframe de "SGames" y unir con la tabala anterior:

In [109]:
# Crear subconjunto de "SGames":
Games_subset= SGames[['developer','id']]

#Unir con la tabla "UReviews_gb"
End4_BestDeveloperYear= pd.merge(Games_subset, UReviews_gb, left_on='id', right_on='item_id')

# Seleccionar las columnas necesarias:
End4_BestDeveloperYear=End4_BestDeveloperYear[['developer', 'item_id','posted_year','count']]
End4_BestDeveloperYear

,developer,item_id,posted_year,count
0,Valve,70,2011,1
1,Valve,70,2012,2
2,Valve,70,2013,8
3,Valve,70,2014,16
4,Valve,70,2015,11
...,...,...,...,...
3359,"Epic Games, Inc.",13230,2012,1
3360,"Epic Games, Inc.",13230,2014,1
3361,"Epic Games, Inc.",13230,2015,2
3362,Valve,80,2013,1


5.2. Crear función "best_developer_year":

In [114]:
def best_developer_year(año: int):
    # Filtrar End4_BestDeveloperYear por el año dado
    filtered_data = End4_BestDeveloperYear[End4_BestDeveloperYear['posted_year'] == año]

    # Filtrar por juegos más recomendados (recommend = Verdadero y comentarios positivos)
    top_developers = filtered_data[filtered_data['count'] > 0]

    # Sumar los conteos por desarrollador y obtener el ranking de los 3 mejores desarrolladores
    top3_developers = top_developers.groupby('developer')['count'].sum().nlargest(3).reset_index()

    # Crear la lista de resultados en el formato deseado
    result_list = [{"Puesto {}".format(i + 1): developer} for i, developer in enumerate(top3_developers['developer'])]

    return result_list

5.3. Probar función:

In [115]:
# Ejemplo de uso
año_ejemplo = 2015  # Reemplazar con el año deseado
resultado = best_developer_year(año_ejemplo)
print(resultado)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Facepunch Studios'}, {'Puesto 3': 'Smartly Dressed Games'}]


5.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [116]:
# Exportar el DataFrame a CSV:

End4_BestDeveloperYear.to_csv('Data_Queries/End4_BestDeveloperYear.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/End4_BestDeveloperYear.csv'}")

DataFrame exportado a Data_Queries/End4_BestDeveloperYear.csv


### 6. Crear función que mida la percepción por desarrollador:

def desarrollador_reviews_analysis( desarrolladora: str ) : Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

6.1. Crear los datos de entrada:

In [123]:
# crear subconjunto de UReviews con el análisis de sentimiento:
sentimiento=UReviews[['item_id','sentiment_analysis']]

# Unimos data set creutilizando el subconjunto creado de Games_subset:
End5_Sentimiento=pd.merge(Games_subset, sentimiento, left_on='id', right_on='item_id')

# Seleccionamos columnas necesarias:
End5_Sentimiento=End5_Sentimiento[['developer','item_id','sentiment_analysis']]

# Revisamos dataframe:
End5_Sentimiento


,developer,item_id,sentiment_analysis
0,Valve,70,2
1,Valve,70,0
2,Valve,70,0
3,Valve,70,2
4,Valve,70,2
...,...,...,...
44734,Valve,80,1
44735,Valve,80,2
44736,Valve,80,2
44737,Valve,80,1


6.2. Crear función "developer_reviews_analysis":

In [126]:
def developer_reviews_analysis(desarrolladora: str):
    # Filtrar End5_Sentimiento por el desarrollador dado
    developer_data = End5_Sentimiento[End5_Sentimiento['developer'] == desarrolladora]

    # Contar la cantidad de registros con análisis de sentimiento positivo y negativo
    positive_count = (developer_data['sentiment_analysis'] == 2).sum()
    negative_count = (developer_data['sentiment_analysis'] == 0).sum()

    # Crear el diccionario de resultados
    result_dict = {desarrolladora: {'Negative': negative_count, 'Positive': positive_count}}

    return result_dict

6.3. Probar función:

In [125]:
# Ejemplo de uso
desarrolladora_ejemplo = 'Valve'  # Reemplazar con el nombre del desarrollador deseado
resultado = developer_reviews_analysis(desarrolladora_ejemplo)
print(resultado)

{'Valve': {'Negative': 901, 'Positive': 5379}}


6.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [127]:
# Exportar el DataFrame a CSV:

End5_Sentimiento.to_csv('Data_Queries/End5_Sentimiento.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/End5_Sentimiento.csv'}")

DataFrame exportado a Data_Queries/End5_Sentimiento.csv
